<a href="https://colab.research.google.com/github/Anas-Altaf/Gdrive_Torrent_Downloader/blob/main/Tor_to_Gdrive_with_Bar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torrent To Google Drive Downloader

**Important Note:** To get more disk space:
> Go to Runtime -> Change Runtime and set GPU as the Hardware Accelerator. You will get around 384GB to download any torrent you want.

### Install libtorrent and Initialize Session

In [1]:
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

### Mount Google Drive
To stream files we need to mount Google Drive.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

### Add From Torrent File
You can run this cell to add more files as many times as you want

In [3]:
from google.colab import files

source = files.upload()
for filename in source.keys():
    ti = lt.torrent_info(filename)
    h = ses.add_torrent({'ti': ti, 'save_path': '/content/drive/MyDrive/Torrents'})
    downloads.append(h)
    print(f'Added torrent: {filename}')

### Add From Magnet Link
You can run this cell to add more files as many times as you want

In [4]:
params = {'save_path': '/content/drive/My Drive/Torrents'}

while True:
    magnet_link = input('Enter Magnet Link Or Type Exit: ')
    if magnet_link.lower() == 'exit':
        break
    h = lt.add_magnet_uri(ses, magnet_link, params)
    downloads.append(h)
    print('Added magnet link')

### Start Download
Monitor the download progress

In [5]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    'queued',
    'checking',
    'downloading metadata',
    'downloading',
    'finished',
    'seeding',
    'allocating',
    'checking fastresume'
]

layout = widgets.Layout(width='auto')
style = {'description_width': 'initial'}
download_bars = [
    widgets.FloatSlider(step=0.01, disabled=True, layout=layout, style=style)
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()
            bar.description = f'{download.name()} {s.download_rate / 1000:.2f} kB/s {state_str[s.state]}'
            bar.value = s.progress * 100
        else:
            downloads.remove(download)
            download_bars.remove(bar)
            next_shift -= 1
    time.sleep(1)